In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# read in all tracking data
df = pd.concat([pd.read_csv('tracking2018.csv.zip'),
                    pd.read_csv('tracking2019.csv.zip'),
                    pd.read_csv('tracking2020.csv.zip')],
                    ignore_index=True)
df

In [ ]:
# determine possible events
df['event'].unique()

In [ ]:
# whittle down tracking data to only show the kicker at time of field goal attempt and at time of made field goal
df = df[((df['event']=='field_goal') | (df['event']=='field_goal_attempt')) & (df['position']=='K')]
df

In [ ]:
# create a globally unique playID
df['compositePlayId'] = df['gameId'].astype(str) + '-' + df['playId'].astype(str)
df

In [ ]:
# add an identifier to note whether a field goal attempt was successful or not
# whittle down tracking data to only the moment of field goal attempts

madeFieldGoals = df[df['event']=='field_goal']['compositePlayId'].tolist()

df = df[df['event']=='field_goal_attempt'].reset_index(drop=True)

df['made'] = np.where(df['compositePlayId'].isin(madeFieldGoals),1,0)

df

In [ ]:
# create a year column for easy filtering
df['year'] = df['time'].apply(lambda x: x[:4])
df

In [ ]:
# adjust tracking coordinates to match the 3d football field made earlier in matplotlib
# depending on direction, we need to change x's origin to be one of the goal lines
# and we need to change y's origin to be the center of the field
# y also needs a compensatory figure (1 or 1.25, chosen very arbritarily by observation of limited film) to
# offset the distance from the kicker to the ball; we also cap the distance to be the hash mark boundary of +-3.083
# we also need to switch x and y

df['x'] = np.where(df['dir'] >= 180, df['x'] - 10, 110 - df['x'])
df['y'] = np.where(df['dir'] >= 180, (53.3 / 2) - 1.25 - df['y'], (53.3 / 2) + 1.5 - df['y'])
df['y'] = np.clip(df['y'], a_max=3.25, a_min=-3.25)
df['x'], df['y'] = df['y'], df['x']
df

In [ ]:
# weed out kicks that have an incorrect y distance
# these are known by the fact that the dis and o columns are vastly different from one another
# and the kicks are impossibly long
# we could alternatively keep them in the dataset and correct them
# but I'm worried about data quality, and there are only a handful of these kicks

df = df[~((df['y']>50) & (abs(df['o'] - df['dir']) > 150))]
df

In [ ]:
# only retain columns we need
df = df[['x','y','made','displayName','jerseyNumber','time','year',]]
df

In [ ]:
# write to csv
df.to_csv('field_goal_coordinates.csv.gz', compression='gzip', index = False)